In [1]:
import pandas as pd
import sklearn_crfsuite
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

import sklearn_crfsuite
from sklearn_crfsuite import metrics


In [2]:
df= pd.read_parquet(r'C:\Users\72118423187\desenvolvimento\github\experiments\members\ian\deep-learning\acts_parquet\iob_aviso_licitacao.parquet')
result = pd.DataFrame(df)

In [3]:
result

,treated_text,IOB
0,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 01/202...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
1,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 50/201...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
2,PREGÃO ELETRÔNICO Nº 14/2021 Processo: 092.041...,B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO ...
3,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO (SRP) Nº ...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
4,AVISO DE LICITAÇÃO TOMADA DE PREÇOS Nº 144/200...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
...,...,...
229,PREGÃO Nº 442/2004 SUCOM/SEF/DF Objeto: Aquisi...,B-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM...
230,PREGÃO Nº 451/2004 SUCOM/SEF/DF Objeto: Aquisi...,B-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM...
231,AVISO DE LICITAÇÃO A COMPANHIA ENERGÉTICA DE B...,O O O O O O O O O O O O O O O O O O O O O O O ...
232,AVISO DE LICITAÇÃO PROCESSO N.º 053.000.918/20...,O O O O O O B-PROCESSO I-PROCESSO I-PROCESSO I...


In [4]:
#
# Transforma as setencas da coluna treated_text em listas de palavras
# Transforma as setencas da coluna IOB me listas de entidades
#
def preparaVetoresXeY(dataset):
    TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize

    x = []
    y = []
    for row in range(len(dataset)):
        if pd.notna(dataset['treated_text'][row]):
            # text_iob = result['IOB'][row].decode("utf-8").replace("', '", '').replace("[' ", '').replace("']", '')
            x.append(TOKENIZER(dataset['treated_text'][row]))
            y.append(dataset['IOB'][row].split())
    return x, y

def tokenizaTextoRaw(texto):
    TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize

    return TOKENIZER(texto)
    

#
# Verifica se o número de palavras da sentença (X) está igual ao numero de anotações de entidade (y)
#
def validaTransformacao(x, y):
    if (len(x) != len(y)):
        print("Erro: o número de sentenças do DODF não correspondem com o número de anotaçôes. x=%i e y=%i" %(len(x), len(y)))
        return
        
    for row in range(len(x)):
        tamanho_x = len(x[row])
        tamanho_y = len(y[row])
        if (tamanho_x != tamanho_y):
            print("Erro: Tamanhos de X e Y no índice=%i deveriam ser iguais. X=%i e Y=%i." %(row, tamanho_x, tamanho_y))
            return

#
# Extrai features das sentencas do DODF
#
def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        for i in range(len(sentence)):
            word_feat = {
                'word': sentence[i].lower(),
                'word[-3:]': sentence[i][-3:],
                'word[-2:]': sentence[i][-2:],
                'capital_letter': sentence[i][0].isupper(),
                'word_istitle': sentence[i].istitle(),
                'all_capital': sentence[i].isupper(),
                'word_isdigit': sentence[i].isdigit(),
                'word_before': '' if i == 0 else sentence[i-1].lower(),
                'word_before_isdigit': '' if i == 0 else sentence[i-1].isdigit(),
                'word_before_isupper': '' if i == 0 else sentence[i-1].isupper(),
                'word_before_istitle': '' if i == 0 else sentence[i-1].istitle(),
                'word_after:': '' if i+1 >= len(sentence) else sentence[i+1].lower(),
                'word_after_isdigit:': '' if i+1 >= len(sentence) else sentence[i+1].isdigit(),
                'word_after_isupper:': '' if i+1 >= len(sentence) else sentence[i+1].isupper(),
                'word_after_istitle:': '' if i+1 >= len(sentence) else sentence[i+1].istitle(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
import math
def repartirDadosTreinoTeste(x,y):
    split=0.8
    split_idx = math.floor(split*len(x))
    x_train = x[0:split_idx]
    y_train = y[0:split_idx]
    x_test = x[split_idx:]
    y_test = y[split_idx:]
    print("Qtd treino = %i, Qtd teste = %i" %(len(x_train),len(x_test)))
    print("%i porcento para Treino" %(split*100))    
    return x_train, y_train, x_test, y_test



#
# Concatena cada palavra do texto do ato com sua respectiva anotação de entidade
#
def concatenaPredicao(ato,  predicao):
    for i in range(len(ato)):
               print(ato[i] + '[' + predicao[i] + ']' , end=" ")

### 1. Prepara o dataset

In [5]:
x_data, y_data = preparaVetoresXeY(result);

### 2.Valida a prepação do dataset

In [6]:
validaTransformacao(x_data, y_data)

### 3. Extrai features das sentencas do DODF

In [7]:
x = []
for i in range(len(x_data)):
    x.append(get_features(x_data[i]))
x_data = x

### 4. Divide dataset em Treino e Teste (hold-out)

In [8]:
 x_train, y_train, x_test, y_test = repartirDadosTreinoTeste(x_data,y_data)

Qtd treino = 187, Qtd teste = 47
80 porcento para Treino


### 5. Treina o modelo

#### Usa parametros default do CRF

In [26]:
crf = sklearn_crfsuite.CRF(
    #algorithm = 'lbfgs'
    #c1=0.17,
    #c2=0.17,
    #max_iterations=1000,
    #all_possible_transitions=True
)

#### Cross-Validation (TODO1: implementar grindsearch para achar melhores valore de c1 e c2)

ver secao Hyperparameter Optimization em:  https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

#Cria-se uma custom scorer com a métrica f1_score macro
custom_scorer_f1 = make_scorer(metrics.flat_f1_score, average='macro')

scores = cross_val_score(crf, x_train, y_train, cv=5, scoring=custom_scorer_f1)

scores

array([0.65020395, 0.72186731, 0.66394081, 0.68682123, 0.6412906 ])

In [23]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.67 accuracy with a standard deviation of 0.03


### 6. Teste 

(TODO2: Usar melhor combinacao de hyperparametros c1 e c2)

In [27]:
#Apos implementar grindseach (TODO1), usar best_estimator
#crf = rs.best_estimator_

crf.fit(x_train, y_train)

y_pred = crf.predict(x_test)

In [25]:
print(metrics.flat_classification_report(
    y_test, y_pred, digits=3, labels=None
))

                          precision    recall  f1-score   support

B-CODIGO_SISTEMA_COMPRAS      0.400     0.333     0.364         6
         B-DATA_ABERTURA      0.818     0.692     0.750        52
  B-MODALIDADE_LICITACAO      1.000     0.854     0.921        48
         B-NUM_LICITACAO      0.906     0.857     0.881        56
         B-OBJ_LICITACAO      0.949     0.822     0.881        45
       B-ORGAO_LICITANTE      1.000     0.267     0.421        15
              B-PROCESSO      0.824     0.667     0.737        42
       B-SISTEMA_COMPRAS      0.789     0.811     0.800        37
              B-TIPO_OBJ      0.000     0.000     0.000         0
        B-VALOR_ESTIMADO      0.875     0.412     0.560        17
I-CODIGO_SISTEMA_COMPRAS      1.000     0.200     0.333         5
         I-DATA_ABERTURA      0.740     0.824     0.780       131
  I-MODALIDADE_LICITACAO      1.000     0.765     0.867        51
         I-NUM_LICITACAO      0.941     0.941     0.941        51
         

### Aplicado modelo em ato

In [15]:
ato1 = "AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONICO No 34/2019. Processo: 001-001334/2019. Objeto: Registro de precos para eventual contratacao de servicos de organizacao de eventos e correlatos, a serem realizados nas dependencias internas e externas da Camara Legislativa do Distrito Federal, sob demanda, abrangendo o planejamento operacional, apoio logistico, organizacao, execucao, montagem de infraestrutura, fornecimento de bens e mao-de-obra, independentemente da prestacao de servicos concomitantes, utilizando mobiliario necessario e adequado, fornecimento de layouts para estandes e exposicoes, compreendendo a montagem, desmontagem, limpeza, manutencao, sonorizacao, iluminacao, instalacoes eletrica e equipamentos, conforme os requisitos e condicoes estabelecidos no Termo de Referencia - Anexo I, do Edital. Valor estimado: R$ 6.014.928,12. Data/hora da Sessao Publica: 25/10/2019. Local: Internet, no endereco www.comprasnet.gov.br. Tipo: menor preco global. O edital encontra-se nos enderecos: www.comprasnet.gov.br (UASG 974004) e www.cl.df.gov.br, no link transparencia. Maiores informacoes (61) 3348-8650. Brasilia/DF, 14 de outubro de 2019. DOUGLAS BRUNO GONCALVES DE LAET."
ato2 = "AVISO DE TOMADA DE PREÇOS Nº 01/2021 Processo 00220-00000503/2021-51 O DISTRITO FEDERAL, por meio da SECRETARIA DE ESTADO DE ESPORTE E LAZER DO DISTRITO FEDERAL, autorizada pelo Decreto Distrital nº 41.497/2020, por meio da Comissão Permanente de Licitação, doravante denominada CPL/SEL/GDF, mediante a Portaria nº 186, de 27 de novembro de 2020, publicada no DODF nº 225, de 01 de dezembro de 2020, torna público, para conhecimento dos interessados, que fará licitação, cujo objeto é: CONTRATAÇÃO DE EMPRESA ESPECIALIZADA EM LIMPEZA, NIVELAMENTO E EXECUÇÃO DE PISO EM CONCRETO ARMADO dos terrenos onde serão implantados os módulos esportivos, objeto do processo 00220-00000398/2021-50, sob a modalidade de TOMADA DE PREÇOS, Tipo MENOR PREÇO GLOBAL POR LOTE, Adjudicação POR LOTE. VALOR TOTAL ESTIMADO: R$ 1.348.175,68(um milhão, trezentos e quarenta e oito mil, cento e setenta e cinco reais e sessenta e oito centavos), em SESSÃO PÚBLICA: no dia 03 de agosto de 2021, às 13h00min, DATA LIMITE DE RECEBIMENTO DOS DOCUMENTOS DE HABILITAÇÃO E PROPOSTA DE PREÇOS: até às 13:00 horas do dia 03 de agosto de 2021, conforme especificações e quantitativos constantes no Edital. O Edital encontra-se disponibilizado, sem ônus, no setor de Comissão Permanente de Licitação, doravante denominada CPL/SEL/GDF no endereço SCS, Quadra 4, Edifício Luiz Carlos Botelho, 7º andar - Bairro Asa Sul – Brasília/DF; CEP: 70.304-000, e na página da Secretaria de Estado de Esporte e Lazer do Distrito Federal – SEL/GDF, www.esporte.df.gov.br (clicar em Publicações). THIAGO MOREIRA CARVALHO DOS SANTOS Presidente da Comissão"
ato3 = "AVISO DE ABERTURA PREGÃO ELETRÔNICO POR SRP Nº 33/2022 - UASG 926119 Objeto: Aquisição de material médico hospitalar CATETER INTRAVENOSO PERIFÉRICO 14G e outros, para atender a demanda da Secretaria de Saúde – DF, conforme especificações e quantitativos constantes no Anexo I do Edital. Processo 00060-00437454/2021-51. Total de 37 itens (Ampla concorrência e cotas destinadas às ME/EPPs). Valor Estimado: R$ 7.833.355,042. Cadastro das Propostas: a partir de 31/01/2022. Abertura das Propostas 10/02/2022, às 9h, horário de Brasília, no site www. net.gov.br. O Edital encontra-se disponibilizado, sem ônus, no site, ou, com ônus, no endereço: SRTVN, Quadra 701, Conjunto C, Edifício PO 700, 2º andar, sala: Central de Compras/DAQ/SUAG, CEP: 70.723-040 – Brasília/DF. PEDRO PAULO B.D.C. FLEURY Pregoeiro"
ato4 = "AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 18/2022 - UASG 974002 A Pregoeira comunica aos interessados que a fim de atender a demanda do Jardim Botânico e Brasília - JBB, a Subsecretaria de Compras Governamentais - SCG/SPLAN/SEEC operacionalizará licitação do PE nº 18/2022 no sistema Comprasnet, cujo objeto é a aquisição de bomba com vazão, bomba hidráulica, motobomba, bloco de cimento, cimento, relê, disjuntor, mini contactor, etc, conforme especificações e condições estabelecidas no termo de referência constante do Anexo I do Edital. Valor total estimado: R$ 46.237,77. Tipo de Licitação: Menor Preço. Unidade Orçamentária: 150106; Programa de Trabalho: 18.122.8210.8517.9658. Elemento de despesa: 33.90.30 e 44.90.52. Fonte: 100 e 120. Abertura das propostas dia 21/02/2022, às 9h30min. Processo 00195-00000275/2021-73. O edital poderá ser retirado no endereço eletrônico: www.comprasgovernamentais.gov.br. Informações pelo e-mail: pregoeirosulog08@economia.df.gov.br Brasília/DF, 04 de fevereiro de 2022 RITA DE CÁSSIA GODINHO DE CAMPOS"
ato5 = "AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 06/2022 Objeto: Contratação de empresa especializada para fornecimento de 13 (treze) notebooks de alta performance e prestação de garantia on site de 12 (doze) meses, para atendimento das demandas internas do Tribunal de Contas do Distrito Federal. Processo: 00600-00000249/2021-80-TCDF. Valor estimado: R$ 155.654,46; enquadramento: natureza 4.4.90.52.35 – Equipamentos e Material Permanente; classificação funcional e programática 01.126.8231.1471.0005 - Modernização de Sistema de Informação do TCDF; fonte de Recursos 100. Data limite de recebimento das propostas: 17/02/2022, às 14h30min. Cópia do Edital encontra-se à disposição no Serviço de Licitação, localizado no 2º Andar do Ed. Anexo do TCDF, telefone (61) 3314- 2742 ou pelos sítios: www.tc.df.gov.br e www.comprasnet.gov.br (UASG: 974003). A Sessão Pública será processada no sítio do ComprasNet, nos termos do Edital. Em atendimento à Lei Distrital nº 5.453/2015, as informações referentes ao certame também estão disponíveis no sítio www.tc.df.gov.br, link: Consulta Processo do TCDF. Brasília/DF, 02 de fevereiro de 2022 ALESSANDRA RIBEIRO ASTUTI Pregoeira"

atos_lista = [ato1, ato2, ato3, ato4, ato5]

atos_tokens_lista = [tokenizaTextoRaw(atos_lista[i]) for i in range(len(atos_lista))] 

atos_features_lista = [get_features(atos_tokens_lista[i]) for i in range(len(atos_tokens_lista))]
    
predicao = crf.predict(atos_features_lista)


### Resultado da predição

In [17]:
concatenaPredicao(atos_tokens_lista[3], predicao[3])

AVISO[O] DE[O] LICITAÇÃO[O] PREGÃO[B-MODALIDADE_LICITACAO] ELETRÔNICO[I-MODALIDADE_LICITACAO] Nº[O] 18[B-NUM_LICITACAO] 2022[I-NUM_LICITACAO] UASG[O] 974002[O] A[O] Pregoeira[O] comunica[O] aos[O] interessados[O] que[O] a[O] fim[O] de[O] atender[O] a[O] demanda[O] do[O] Jardim[O] Botânico[O] e[O] Brasília[O] JBB[O] a[O] Subsecretaria[O] de[O] Compras[O] Governamentais[O] SCG[O] SPLAN[O] SEEC[O] operacionalizará[O] licitação[O] do[O] PE[O] nº[O] 18[B-NUM_LICITACAO] 2022[I-NUM_LICITACAO] no[O] sistema[O] Comprasnet[O] cujo[O] objeto[O] é[O] a[O] aquisição[B-OBJ_LICITACAO] de[I-OBJ_LICITACAO] bomba[I-OBJ_LICITACAO] com[I-OBJ_LICITACAO] vazão[I-OBJ_LICITACAO] bomba[I-OBJ_LICITACAO] hidráulica[I-OBJ_LICITACAO] motobomba[I-OBJ_LICITACAO] bloco[I-OBJ_LICITACAO] de[I-OBJ_LICITACAO] cimento[I-OBJ_LICITACAO] cimento[I-OBJ_LICITACAO] relê[I-OBJ_LICITACAO] disjuntor[I-OBJ_LICITACAO] mini[I-OBJ_LICITACAO] contactor[I-OBJ_LICITACAO] etc[I-OBJ_LICITACAO] conforme[I-OBJ_LICITACAO] especificações[I-OBJ

In [80]:
#def getModalidadeLicitacao():
import numpy as np


def resultadoModelo(listaEntidades):
    
    if (listaEntidades == 'OBJ_LICITACAO'):
        values = np.array(predicao[3])

        searchval = 'I-OBJ_LICITACAO'

        ii = np.where(values == searchval)[0]
        
        res_list = [atos_tokens_lista[3][i] for i in ii.tolist()]
        
        for i in res_list: 
            print(i, end = ' ')



resultadoModelo('OBJ_LICITACAO')




de bomba com vazão bomba hidráulica motobomba bloco de cimento cimento relê disjuntor mini contactor etc conforme especificações e condições estabelecidas no termo de referência constante do Anexo I do Edital 

In [72]:
res_list = [atos_tokens_lista[3][i] for i in ii.tolist()]

In [73]:
res_list

['de',
 'bomba',
 'com',
 'vazão',
 'bomba',
 'hidráulica',
 'motobomba',
 'bloco',
 'de',
 'cimento',
 'cimento',
 'relê',
 'disjuntor',
 'mini',
 'contactor',
 'etc',
 'conforme',
 'especificações',
 'e',
 'condições',
 'estabelecidas',
 'no',
 'termo',
 'de',
 'referência',
 'constante',
 'do',
 'Anexo',
 'I',
 'do',
 'Edital']